In [10]:
import pandas as pd
from geopy.distance import geodesic

# Função para calcular a distância entre duas coordenadas
def calcular_distancia(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).kilometers

# Carregar o arquivo CSV com as estações
df_novo = pd.read_csv('Dados_Bacia_8_Subestacao_86_Ativas.csv', sep=';')
df_novo = df_novo[df_novo['Data_Periodo_Telemetrica_Inicio'] <= '2019-01-01']
df_novo = df_novo[df_novo['Tipo_Estacao_Telemetrica'] == 1]

# Filtrar as estações com valores válidos de latitude, longitude e que são pluviométricas
df_novo_filtrado = df_novo.dropna(subset=['Latitude', 'Longitude'])
#df_novo_filtrado = df_novo_filtrado[df_novo_filtrado['Tipo_Estacao'] == 'Fluviometrica']
df_novo_filtrado = df_novo_filtrado[df_novo_filtrado['Tipo_Estacao'] == 'Pluviometrica']

# Definir o critério de altitude mínima para considerar as estações na serra (altitude > 250 metros)
altitude_minima = 200
df_novo_filtrado_serra = df_novo_filtrado[df_novo_filtrado['Altitude'] > altitude_minima]

# Calcular a média de latitude e longitude para determinar a estação mais central
centro_latitude = df_novo_filtrado_serra['Latitude'].mean()
centro_longitude = df_novo_filtrado_serra['Longitude'].mean()

# Calcular a distância de todas as estações até o ponto central e escolher a mais próxima
df_novo_filtrado_serra['distancia_centro'] = df_novo_filtrado_serra.apply(
    lambda row: calcular_distancia(row['Latitude'], row['Longitude'], centro_latitude, centro_longitude), axis=1)

# Selecionar a estação mais próxima do centro
estacao_central = df_novo_filtrado_serra.loc[df_novo_filtrado_serra['distancia_centro'].idxmin()]

# Lista de estações selecionadas, começando com a central
selecionadas_serra = [estacao_central['codigoestacao']]

# Criar uma matriz de distâncias entre as estações
distancias_serra = pd.DataFrame(index=df_novo_filtrado_serra['codigoestacao'], columns=df_novo_filtrado_serra['codigoestacao'])

# Calcular a distância entre todas as estações filtradas
for i, est1 in df_novo_filtrado_serra.iterrows():
    for j, est2 in df_novo_filtrado_serra.iterrows():
        distancias_serra.loc[est1['codigoestacao'], est2['codigoestacao']] = calcular_distancia(est1['Latitude'], est1['Longitude'], est2['Latitude'], est2['Longitude'])

# Converter a matriz de distâncias para valores numéricos
distancias_serra = distancias_serra.apply(pd.to_numeric)

# Selecionar mais 13 estações de forma homogênea, maximizando a menor distância a cada passo
for _ in range(20):
    # Calcular a menor distância de cada estação em relação às já selecionadas
    dist_minimas_serra = distancias_serra.loc[selecionadas_serra].min(axis=0)
    # Selecionar a estação que tem a maior distância mínima, para distribuir melhor as estações
    proxima_estacao_serra = dist_minimas_serra.drop(selecionadas_serra).idxmax()
    selecionadas_serra.append(proxima_estacao_serra)

# Obter os nomes e altitudes das estações selecionadas
df_selecionadas_serra = df_novo_filtrado_serra[df_novo_filtrado_serra['codigoestacao'].isin(selecionadas_serra)][['codigoestacao', 'Estacao_Nome', 'Latitude', 'Longitude', 'Altitude']]

# Exibir os nomes e os códigos das estações selecionadas
print(df_selecionadas_serra[['codigoestacao','Estacao_Nome' ]])


C:\Users\fabri\AppData\Local\Temp\ipykernel_21684\4059716119.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_novo_filtrado_serra['distancia_centro'] = df_novo_filtrado_serra.apply(


     codigoestacao                         Estacao_Nome
2          2850009                        PASSO TAINHAS
7          2951120               NOVA PETROPOLIS_Centro
24         2951109   CAXIAS DO SUL_Parque dos Pinheiros
27         2951087                UHE CASTRO ALVES ALÇA
33         2951070                                MUÇUM
40         2951003                      BENTO GONÇALVES
172        2952051        PCH RASTRO DE AUTO BARRAMENTO
181        2950072       PCH CAZUZA FERREIRA BARRAMENTO
186        2852068               FONTOURA XAVIER_Centro
188        2852004                                AULER
189        2852005               CGH CAPIGUI BARRAMENTO
205        2851067                  PCH BOA FÉ MONTANTE
207        2851069           PCH SANTA CAROLINA JUSANTE
209        2851072                           IBIRAIARAS
221        2851044                              GUAPORÉ
237        2850050               CGH CAMBARÁ BARRAMENTO
239        2850039       PCH PEZZI ARROIO PINHEI

In [12]:
df_selecionadas_serra[['codigoestacao','Estacao_Nome' ]]


,codigoestacao,Estacao_Nome
2,2850009,PASSO TAINHAS
7,2951120,NOVA PETROPOLIS_Centro
24,2951109,CAXIAS DO SUL_Parque dos Pinheiros
27,2951087,UHE CASTRO ALVES ALÇA
33,2951070,MUÇUM
40,2951003,BENTO GONÇALVES
172,2952051,PCH RASTRO DE AUTO BARRAMENTO
181,2950072,PCH CAZUZA FERREIRA BARRAMENTO
186,2852068,FONTOURA XAVIER_Centro
188,2852004,AULER


In [11]:
import folium

# Criar o mapa centrado na média das coordenadas das estações selecionadas
centro_latitude = df_selecionadas_serra['Latitude'].mean()
centro_longitude = df_selecionadas_serra['Longitude'].mean()
mapa = folium.Map(location=[centro_latitude, centro_longitude], zoom_start=8)

# Adicionar as estações ao mapa
for _, estacao in df_selecionadas_serra.iterrows():
    folium.Marker(
        location=[estacao['Latitude'], estacao['Longitude']],
        popup=f"Estação: {estacao['Estacao_Nome']},Código:{estacao['codigoestacao']}, Altitude: {estacao['Altitude']} m",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(mapa)

# Salvar o mapa em um arquivo HTML
mapa.save('mapa_estacoes_pluviometricas_central_distribuido.html')

# Exibir uma mensagem para informar que o mapa foi gerado
print("Mapa gerado e salvo como 'mapa_estacoes_pluviometricas_central_distribuido.html'. Abra este arquivo no seu navegador para visualizar.")


Mapa gerado e salvo como 'mapa_estacoes_pluviometricas_central_distribuido.html'. Abra este arquivo no seu navegador para visualizar.
